Background sample : 각 class 별 10%

street-facilities-2  
class 0, 1, 2 (the counts of each object class, not image counts)  
[2698 7672 5397]  

X4, X1.5 X2  

10792, 11508, 10794  

X8, X3 X4  
21584, 23016, 21588  

Negative sample files 갯수 비교

In [ ]:
import os

root_path = "D:/Downloads/street-facilities-selected"
json_path = os.path.join(root_path, "labels")

labels = [f for f in os.listdir(json_path) if f.endswith(".json")]
total_cnt = len(labels)

negative_sample_cnt = [0, 0, 0]
type_cnt_mapping = {
    9 : 0,
    12 : 1,
    13 : 2,
}

for label in labels:
    type = int(label.split("_")[1]) # 09 or 12 or 13
    state = int(label.split("_")[2]) # 0 정상 1 불량
    if state == 0:
        negative_sample_cnt[type_cnt_mapping[type]] += 1

print("total : ", total_cnt)
print("negative examples : ", negative_sample_cnt)

total :  8477
negative examples :  [182, 1083, 915]


### Image rotation

In [51]:
import numpy as np
import cv2
import matplotlib.pyplot as plt


def clip_bboxes(bboxes : np.array):
    # The function that clips the bounding box exceeding 0-1 coordinate range in the normalized coordinate plane
    # normalized 된 좌표상에서 0~1 범위를 벗어난 bounding box 에 대해
    # clipping 하여 0~1 범위 내로 보정하는 함수
    clipped_bboxes = []
    for bbox in bboxes:
        class_type = bbox[0]
        xcenter, ycenter, width, height = bbox[1:]
        xmin, xmax = xcenter - width/2, xcenter + width/2
        ymin, ymax = ycenter - height/2, ycenter + height/2
        
        xmin = 0.0 if xmin < 0 else xmin
        xmax = 1.0 if xmax > 1 else xmax
        ymin = 0.0 if ymin < 0 else ymin
        ymax = 1.0 if ymax > 1 else ymax

        clipped_bboxes.append([
            class_type,
            (xmin + xmax) / 2,
            (ymin + ymax) / 2,
            (xmax - xmin),
            (ymax - ymin),
        ])
        
    return np.array(clipped_bboxes, ndmin=2)

# bbox 를 이미지 위에 그리기
def visualize_bbox(image, bboxes, rgb: tuple = (0, 255, 0), thickness : int = 3):
    for bbox in bboxes:
        class_type = bbox[0]
        xcenter, ycenter, width, height = bbox[1:]
        xmin, xmax = xcenter - width/2, xcenter + width/2
        ymin, ymax = ycenter - height/2, ycenter + height/2
        
        h, w, _ = image.shape
        pt1 =  np.int16(np.ceil([xmin * w, ymin * h]))
        pt2 = np.int16(np.ceil([xmax * w, ymax * h]))

        image = cv2.rectangle(image, pt1, pt2, rgb, thickness)

    plt.figure()
    plt.imshow(image)

In [14]:
import os
import numpy as np
import cv2
import albumentations as A
import matplotlib.pyplot as plt
import warnings # to ignore np.loadtxt warning

root_path = "D:/Downloads/street-facilities-selected"
image_path = os.path.join(root_path, "images")
label_path = os.path.join(root_path, "labels-txt")

save_image_path = os.path.join(root_path, "rotated-images")
save_label_path = label_path

image_names = [f for f in os.listdir(image_path) 
                 if f.endswith('.jpg') or f.endswith('.jpeg')] # .jpeg 도 극히 일부 존재

# test
# image_names = ["3_12_1_1_4_1_20210908_0000529500.jpg"]
# image_names = ["3_12_1_1_2_2_20211108_0001051465.jpg"]

total_cnt = len(image_names)

# ex "3_12_1_1_4_1_20210908_0000529500.jpg" -> the class type is 12
type_state_factor = {
    9 : {
        0 : 9, # type 9 and state 0 (정상) -> 증강 9회 수행 (총 X10)
        1 : 7 # type 9 and state 1 (불량) -> 증강 7회 수행 (X8)
    },
    12: {
        0 : 1, # X2
        1 : 2 # X3
    },
    13 : {
        0 : 1, # X2
        1 : 3, # X4
    }
}

# Albumentations Rotate 변환 설정
# ref.https://albumentations.ai/docs/getting_started/bounding_boxes_augmentation/
# Keypoints are rotated around the center of the image.
# Bounding boxes are rotated and may change size or shape.
# Each point (x, y) in the image is transformed to (x', y') 
# by: [x'] [cos(θ) -sin(θ)][x - cx] [cx] [y'] = [sin(θ) cos(θ)][y - cy] + [cy] where (cx, cy) is the center of the image.
transform = A.Compose([
    A.SafeRotate(limit=90, # +- degrees. Range from which a random angle is picked.
                 border_mode=cv2.BORDER_REPLICATE, 
                 p=1.0, # Probability of applying the transform. Default: 0.5.
                 )], bbox_params=A.BboxParams('yolo'))


for i, image_name in enumerate(image_names):
    if (i % 300 == 0):
        print(f"{i+1}/{total_cnt}")
        
    # 이미지 로드
    image = cv2.imread(os.path.join(image_path, image_name))
    # image = cv2.cvtColor(image, cv2.COLOR_BGR2RGB)

    f_base = os.path.splitext(image_name)[0]
    class_type = int(f_base.split("_")[1]) # [9 or 12 or 13]
    state = int(f_base.split("_")[2]) # 0 정상 1 불량
    label_file = os.path.join(label_path, f_base + ".txt")

    # 빈 txt 파일일 경우 뜨는 numpy 경고문 무시
    with warnings.catch_warnings():
        warnings.simplefilter("ignore") 
        ##### ** 중요 ** .json 파일로부터 작성된 labels-txt 원본 (회전 이전) 이 있어야 함 - format_translation.ipynb
        bboxes = np.loadtxt(fname=label_file, delimiter=" ", ndmin=2)

    # 회전 이전 bbox 시각화
    # visualize_bbox(image, bboxes, (255,0,0))

    # albumentations 처리 형태로 변환
    bboxes = np.roll(bboxes, 4, axis=1)

    # 변환 적용
    # type 과 state 값 마다 반복 수 다름 
    for iter in range(type_state_factor[class_type][state]):
        # 변환
        transformed = transform(image=image, bboxes=bboxes)
        # 변환 결과
        rotated_image = transformed["image"]
        rotated_bboxes = transformed['bboxes']

        # albumentations 처리 형태에서 yolo label 형태로 다시 변환
        rotated_label = np.roll(rotated_bboxes, 1, axis=1)
        # print(rotated_label)

        # 0~1 범위를 벗어나는 좌표값 clipping
        # SafeRotation 을 쓰고 있기 때문에 필요없음
        # rotated_label = clip_bboxes(rotated_label)
        
        # 회전 이후 bbox 시각화
        # visualize_bbox(rotated_image, rotated_label)

        cv2.imwrite(os.path.join(save_image_path, f_base + f"-{iter+1}.jpg"), rotated_image)
        np.savetxt(os.path.join(save_label_path, f_base + f"-{iter+1}.txt"), rotated_label, delimiter=" ", fmt='%.8f')

        # test
        # cv2.imwrite(f_base + f"-{iter+1}.jpg", rotated_image)
        # np.savetxt(f_base + f"-{iter+1}.txt", rotated_label, delimiter=" ", fmt='%.8f')
        # break
    # break

1/8477
301/8477
601/8477
901/8477
1201/8477
1501/8477
1801/8477
2101/8477
2401/8477
2701/8477
3001/8477
3301/8477
3601/8477
3901/8477
4201/8477
4501/8477
4801/8477
5101/8477
5401/8477
5701/8477
6001/8477
6301/8477
6601/8477
6901/8477
7201/8477
7501/8477
7801/8477
8101/8477
8401/8477


classId 는 Int 형이여야 하므로 이에 대한 처리 수행

In [16]:
import os

root_path = "D:/Downloads/street-facilities-selected"

label_path = f"{root_path}/labels-txt"

labels = [label for label in os.listdir(label_path) if label.endswith('.txt')]
total_cnt = len(labels)

for i, label in enumerate(labels):
    if i % 1000 == 0 :
        print(f"{i+1} / {total_cnt}")
    annotations = []
    with open(os.path.join(label_path, label), "r+", encoding="utf-8") as f:
        # 기존 내용 읽기 및 수정
        for line in f:
            elems = line.split(" ")
            elems[0] = str(int(float(elems[0]))) # 1.00000000 -> 1 -> '1'
            newline = " ".join(elems)
            annotations.append(newline)
            
        # 새 내용 작성
        f.seek(0) # 파일 포인터 이동
        for annotation in annotations:
            f.write(annotation)
        f.truncate() # 파일 포인터 이전 내용만 남기고 나머지 삭제

1 / 32610
1001 / 32610
2001 / 32610
3001 / 32610
4001 / 32610
5001 / 32610
6001 / 32610
7001 / 32610
8001 / 32610
9001 / 32610
10001 / 32610
11001 / 32610
12001 / 32610
13001 / 32610
14001 / 32610
15001 / 32610
16001 / 32610
17001 / 32610
18001 / 32610
19001 / 32610
20001 / 32610
21001 / 32610
22001 / 32610
23001 / 32610
24001 / 32610
25001 / 32610
26001 / 32610
27001 / 32610
28001 / 32610
29001 / 32610
30001 / 32610
31001 / 32610
32001 / 32610


변형 없는 원본 이미지 복사 및 붙여넣기

In [20]:
import shutil
import os

root_path = "D:/Downloads/street-facilities-selected"

image_path = os.path.join(root_path, "images")
dpath = os.path.join(root_path, "rotated-images")

images = [image for image in os.listdir(image_path) if image.endswith(".jpg") or image.endswith(".jpeg")]

total_cnt = len(images)

for i, image in enumerate(images):
    if i % 500 == 0:
        print(f"{i+1} / {total_cnt}")
    src = os.path.join(image_path, image)
    dest = os.path.join(dpath, image)
    # 메타데이터 포함 복사
    shutil.copy2(src, dest)  

1 / 8477
501 / 8477
1001 / 8477
1501 / 8477
2001 / 8477
2501 / 8477
3001 / 8477
3501 / 8477
4001 / 8477
4501 / 8477
5001 / 8477
5501 / 8477
6001 / 8477
6501 / 8477
7001 / 8477
7501 / 8477
8001 / 8477


마지막으로 분포 재확인

In [18]:
import numpy as np

label_path = "D:/Downloads/street-facilities-selected/labels-txt"

labels = [label for label in os.listdir(label_path) if label.endswith(".txt")]

file_dist = np.zeros((3,2))

object_dist = np.zeros((3,2))

map = {
    9 : 0,
    12 : 1,
    13 : 2,
}

total_cnt = len(labels)

for i, label in enumerate(labels):
    if i % 500 == 0:
        print(f"{i+1} / {total_cnt}")
    # 파일 분포
    subset = label.split("_")
    ctype = int(subset[1])
    state = int(subset[2])

    file_dist[map[ctype]][state] += 1

    # 오브젝트 분포
    with open(os.path.join(label_path, label), "r+", encoding="utf-8") as f:
        for line in f:
            elems = line.split(" ")
            object_dist[int((elems[0]))][1] += 1

object_dist[:, 0] = file_dist[:, 0]

# 행번호 : class type
# 열번호 : 0-> 정상, 1->불량(파손)

print("파일 분포")
print(file_dist)

print("오브젝트 분포")
print(object_dist)


파일 분포
[[1820. 8952.]
 [2166. 8610.]
 [1830. 9232.]]
오브젝트 분포
[[ 1820. 21584.]
 [ 2166. 23016.]
 [ 1830. 21588.]]


In [19]:
a = np.array([[1820, 8952],
              [2166, 8610],
              [1830, 9232],])

b = np.sum(a, axis=0)
b



array([ 5816, 26794])